In [51]:
import pandas as pd
import requests
import json

class ScriptData:
    def __init__(self):
        self.data = {}
    
    def fetch_intraday_data(self, script):
        # API Key value from link
        api_key = 'GF0IU3N4VNWYWAV5'
        
        # construct the API URL
        base_url = "https://www.alphavantage.co/query"
        function = "TIME_SERIES_INTRADAY"
        symbol = script
        interval = "1min"
        outputsize = "compact"
        api_url = f"{base_url}?function={function}&symbol={symbol}&interval={interval}&outputsize={outputsize}&apikey={api_key}"
        
        # send the request and parse the response
        response = requests.get(api_url)
        if response.status_code != 200:
            raise Exception("Failed to fetch data. Check the API key and script name.")
        data = response.json()
        self.data[script] = data
    
    def convert_intraday_data(self, script):
        # get the data for the given script
        data = self.data[script]
        
        # extract the time series data
        time_series_data = data["Time Series (1min)"]
        
        # create a list of tuples containing the timestamp and data for each time point
        data_list = [(timestamp, data) for timestamp, data in time_series_data.items()]
        
        # sort the data by timestamp
        data_list.sort(key=lambda x: x[0])
        
        # extract the data into separate lists
        timestamps, data = zip(*data_list)
        opens, highs, lows, closes, volumes = [], [], [], [], []
        for d in data:
            opens.append(float(d["1. open"]))
            highs.append(float(d["2. high"]))
            lows.append(float(d["3. low"]))
            closes.append(float(d["4. close"]))
            volumes.append(int(d["5. volume"]))
        
        # create a pandas DataFrame from the lists
        df = pd.DataFrame({"timestamp": timestamps, "open": opens, "high": highs, "low": lows, "close": closes, "volume": volumes})
        
        # set the timestamp column as the index of the DataFrame
        df.set_index("timestamp", inplace=True)
        
        # convert the timestamp strings to pandas Timestamps
        df.index = pd.to_datetime(df.index)
        
        # store the DataFrame in the data dictionary
        self.data[script] = df
    
    def __getitem__(self, key):
        return self.data[key]
    
    def __setitem__(self, key, value):
        self.data[key] = value
    
    def __contains__(self, key):
        return key in self.data


In [52]:
# create an instance of the ScriptData class
script_data = ScriptData()

# fetch intraday data for the 'GOOGL' script using your Alpha Vantage API key
script_data.fetch_intraday_data('GOOGL')

# convert the fetched data into a pandas DataFrame
script_data.convert_intraday_data('GOOGL')

# access the DataFrame for the 'GOOGL' script
df = script_data['GOOGL']




In [53]:
df

,open,high,low,close,volume
timestamp,,,,,
2022-12-23 15:18:00,89.38,89.450,89.3750,89.3900,42515
2022-12-23 15:19:00,89.39,89.430,89.3400,89.4285,30024
2022-12-23 15:20:00,89.42,89.425,89.3350,89.3450,87836
2022-12-23 15:21:00,89.35,89.375,89.2818,89.3050,32896
2022-12-23 15:22:00,89.30,89.305,89.2600,89.2800,23762
...,...,...,...,...,...
2022-12-23 19:35:00,89.09,89.090,89.0900,89.0900,140
2022-12-23 19:48:00,89.11,89.110,89.0600,89.0600,377
2022-12-23 19:54:00,89.06,89.060,89.0600,89.0600,1127


In [54]:

script_data.fetch_intraday_data('AAPL')

script_data.convert_intraday_data('AAPL')

df = script_data['AAPL']





In [55]:
df

,open,high,low,close,volume
timestamp,,,,,
2022-12-23 17:04:00,131.7200,131.7300,131.7200,131.7300,1943
2022-12-23 17:05:00,131.7300,131.7300,131.7300,131.7300,1084
2022-12-23 17:07:00,131.7116,131.7116,131.7116,131.7116,332
2022-12-23 17:09:00,131.7200,131.7200,131.7100,131.7200,3203
2022-12-23 17:10:00,131.7200,131.7200,131.7200,131.7200,331
...,...,...,...,...,...
2022-12-23 19:55:00,131.7400,131.7400,131.7400,131.7400,609
2022-12-23 19:56:00,131.7400,131.7400,131.7400,131.7400,581
2022-12-23 19:57:00,131.7499,131.7499,131.7100,131.7300,3295


In [56]:
'GOOGL' in script_data

True

In [57]:
'AAPL' in script_data

True

In [58]:
'NVDA' in script_data

False

In [70]:
def indicator1(df, timeperiod):
    # create a new column in the DataFrame with the moving average of the close price
    df["indicator"] = df["close"].rolling(timeperiod).mean()
    
    # create a new DataFrame with the timestamp and indicator columns
    result = pd.DataFrame({"timestamp": df.index, "indicator": df["indicator"]})

    return result


In [71]:
# fetch and convert intraday data for the 'GOOGL' script
script_data = ScriptData()
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')

# compute the indicator1 for the 'GOOGL' script with a timeperiod of 5
indicator1_df1 = indicator1(script_data['GOOGL'], timeperiod = 5)


In [72]:
indicator1_df1

,timestamp,indicator
timestamp,,
2022-12-23 15:18:00,2022-12-23 15:18:00,NaN
2022-12-23 15:19:00,2022-12-23 15:19:00,NaN
2022-12-23 15:20:00,2022-12-23 15:20:00,NaN
2022-12-23 15:21:00,2022-12-23 15:21:00,NaN
2022-12-23 15:22:00,2022-12-23 15:22:00,89.3497
...,...,...
2022-12-23 19:35:00,2022-12-23 19:35:00,89.1300
2022-12-23 19:48:00,2022-12-23 19:48:00,89.1240
2022-12-23 19:54:00,2022-12-23 19:54:00,89.1180


In [32]:
script_data = ScriptData()
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')

indicator1_df2 = indicator1(script_data['AAPL'], timeperiod = 5)

In [33]:
indicator1_df2

,timestamp,indicator
timestamp,,
2022-12-23 17:04:00,2022-12-23 17:04:00,NaN
2022-12-23 17:05:00,2022-12-23 17:05:00,NaN
2022-12-23 17:07:00,2022-12-23 17:07:00,NaN
2022-12-23 17:09:00,2022-12-23 17:09:00,NaN
2022-12-23 17:10:00,2022-12-23 17:10:00,131.72232
...,...,...
2022-12-23 19:55:00,2022-12-23 19:55:00,131.73600
2022-12-23 19:56:00,2022-12-23 19:56:00,131.73800
2022-12-23 19:57:00,2022-12-23 19:57:00,131.73800


In [26]:
import pandas as pd

class Strategy:
    def __init__(self,script):
        self.script_data = ScriptData()
    
    def fetch_intraday_data(self, script):
        self.script_data.fetch_intraday_data(script)
        self.script_data.convert_intraday_data(script)
        self.df = self.script_data[script]
    
    def compute_indicator_data(self):
        self.indicator_df = indicator1(self.df, 5)
    
    def generate_signals(self):
        # create a new column in the DataFrame with the moving average of the close price
        self.df["indicator"] = self.indicator_df["indicator"]
        
        # create a new column in the DataFrame with the signals
        self.df["signal"] = "NO_SIGNAL"
        self.df.loc[self.df["indicator"] > self.df["close"], "signal"] = "BUY"
        self.df.loc[self.df["indicator"] < self.df["close"], "signal"] = "SELL"
        
        # create a new DataFrame with the timestamp and signal columns
        self.signals = pd.DataFrame({"timestamp": self.df.index, "signal": self.df["signal"]})


In [34]:
# create an instance of the Strategy class
strategy = Strategy('NVDA')

# fetch and compute data for the 'NVDA' script
strategy.fetch_intraday_data('NVDA')
strategy.compute_indicator_data()

# generate the signals for the 'GOOGL' script
strategy.generate_signals()

# access the signals DataFrame
signals_df = strategy.signals


In [28]:
signals_df

,timestamp,signal
timestamp,,
2022-12-23 15:18:00,2022-12-23 15:18:00,NO_SIGNAL
2022-12-23 15:19:00,2022-12-23 15:19:00,NO_SIGNAL
2022-12-23 15:20:00,2022-12-23 15:20:00,NO_SIGNAL
2022-12-23 15:21:00,2022-12-23 15:21:00,NO_SIGNAL
2022-12-23 15:22:00,2022-12-23 15:22:00,BUY
...,...,...
2022-12-23 19:35:00,2022-12-23 19:35:00,BUY
2022-12-23 19:48:00,2022-12-23 19:48:00,BUY
2022-12-23 19:54:00,2022-12-23 19:54:00,BUY


In [16]:
# filter the signals DataFrame to include only the rows with a 'BUY' or 'SELL' signal
filtered_signals_df = strategy.signals[strategy.signals["signal"].isin(["BUY", "SELL"])]

# print the filtered signals DataFrame
print(filtered_signals_df)


                              timestamp signal
timestamp                                     
2022-12-23 15:22:00 2022-12-23 15:22:00    BUY
2022-12-23 15:23:00 2022-12-23 15:23:00    BUY
2022-12-23 15:24:00 2022-12-23 15:24:00    BUY
2022-12-23 15:25:00 2022-12-23 15:25:00    BUY
2022-12-23 15:26:00 2022-12-23 15:26:00    BUY
...                                 ...    ...
2022-12-23 19:35:00 2022-12-23 19:35:00    BUY
2022-12-23 19:48:00 2022-12-23 19:48:00    BUY
2022-12-23 19:54:00 2022-12-23 19:54:00    BUY
2022-12-23 19:56:00 2022-12-23 19:56:00   SELL
2022-12-23 20:00:00 2022-12-23 20:00:00    BUY

[96 rows x 2 columns]


In [17]:
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc
# create a new DataFrame with the close and indicator data
plot_df = pd.DataFrame({"close": strategy.df["close"], "indicator": strategy.indicator_df["indicator"]})

# plot the candlestick chart
mplfinance.orginal.candlestick_ohlc(plot_df)


NameError: name 'mplfinance' is not defined

In [ ]:

from mplfinance.original_flavor import candlestick_ohlc

In [ ]:
pip install plotly==5.11.0

In [ ]:
import plotly.express as px
# create a new DataFrame with the close and indicator data
plot_df = pd.DataFrame({"close": strategy.df["close"], "indicator": strategy.indicator_df["indicator"]})

# plot the candlestick chart
px.candlestick_ohlc(plot_df)
